In [1]:
import pandas as pd
import plotly.graph_objects as go

In [2]:
full = pd.read_parquet('../data/SO_2014_2022.pq')

In [3]:
df = full.groupby('Gender')['Education'].value_counts()
res = pd.DataFrame()

res['master/doctorate'] = df.loc[:, 'doctor'] + df.loc[:, 'professional'] + df.loc[:, 'master']
res['bachelor/tertiary'] = df.loc[:, 'bachelor'] + df.loc[:, 'tertiary'] + df.loc[:, 'assoc']
res['secondary'] = df.loc[:, 'secondary']
res['primary'] = df.loc[:, 'primary'] + df.loc[:, 'none']
res = res.cumsum(axis=1)
res.loc['male'] *= 1 / res.loc['male']['primary'] * 100
res.loc['female'] *= 1 / res.loc['female']['primary'] * 100
res

,master/doctorate,bachelor/tertiary,secondary,primary
Gender,,,,
female,28.899066,91.328082,99.219637,100
male,27.144707,89.210283,98.916874,100


In [49]:
blue = 'rgba(0,0,255,0.1)'
red = 'rgba(255,0,0,0.1)'

fig = go.Figure(data=[go.Sankey(
    arrangement='snap',
    node={'pad': 15, 'thickness': 20, 'line': {'color': "black", 'width': 0.1},
          'x': [0.01, 0.2, 0.4, 0.6,
                0.01, 0.2, 0.4, 0.6, -1],
          'y': [0.3, 0.3, 0.735, 0.6,
                0.2, 0.2, 0.2, 0.05, 0.1],
          'customdata': ["", "All valid answers from male<br> respondents were used",
                         "Male respondents, secondary degree", "Male respondents, tertiary education",
                         "Male respondents, master / doctorate",
                         "All valid answers from female<br> respondents were used",
                         "Female respondents, secondary degree", "Female respondents, tertiary education",
                         "Female respondents, master / doctorate"],
          'hovertemplate': '%{customdata}<extra></extra>',
          'color': ["green", "#bfc5ff", "#bfc5ff", "#bfc5ff", "#bfc5ff", "pink", "pink", "pink", "pink"]},

    link={'source': [1, 1, 2, 2, 3, 3, 4, 5, 5, 6, 6, 7, 7, 8, 9, 9],
          'target': [2, 0, 3, 0, 4, 0, 0, 6, 0, 7, 0, 8, 0, 0, 1, 5],
          'value': [
              res.secondary['male'], 0,  # res.primary['male'] - res.secondary['male'],
              res['bachelor/tertiary']['male'], 0,  # res.secondary['male'] - res['bachelor/tertiary']['male'],
              res['master/doctorate']['male'], 0,  # res['bachelor/tertiary']['male'] - res['master/doctorate']['male'],
              0,  #res['master/doctorate']['male'],
              res.secondary['female'], 0,  # res.primary['female'] - res.secondary['female'],
              res['bachelor/tertiary']['female'], 0,  # res.secondary['female'] - res['bachelor/tertiary']['female'],
              res['master/doctorate']['female'], 0,
              # res['bachelor/tertiary']['female'] - res['master/doctorate']['female'],
              0,  #res['master/doctorate']['female'],
              100, 100
          ], 'customdata': ["Almost all male respondents<br>completed secondary education", "",
                            "Most male respondents continued to tertiary education,<br>though slightly less than female respondents",
                            "",
                            "Over a quarter of male respondents continued to a master or doctorate,<br>though slightly less than female respondents",
                            "", "", "Almost all female respondents<br>completed secondary education", "",
                            "Most female respondents continued to tertiary education,<br>though slightly more than female respondents",
                            "",
                            "Over a quarter of female respondents continued to a master or doctorate,<br>though slightly more than male respondents", ],
          'hovertemplate': ' %{customdata}<extra></extra>',
          'arrowlen': 14,
          'color': [blue, blue, blue, blue, blue, blue, blue,
                    red, red, red, red, red, red, red,
                    'rgba(255,255,255,0)', 'rgba(255,255,255,0)', ]
          })])

fig.update_layout(title_text="Retention in education system, split by gender", font_size=15, width=1_200, modebar_remove=['lasso2d', 'select2d', 'resetsankeygroup'])

fig.add_annotation(x=0.01, y=1.15,
                   text="Approximately equal, slightly higher for female respondents",
                   font={'color':'grey', 'size':10},
                   showarrow=False)

fig.add_annotation(x=-0.02, y=-0.1,
                   text="All respondents",
                   showarrow=False)

fig.add_annotation(x=0.17, y=-0.1,
                   text="Secondary",
                   showarrow=False)

fig.add_annotation(x=0.4, y=-0.1,
                   text="Bachelor/Tertiary",
                   showarrow=False)

fig.add_annotation(x=0.6, y=-0.1,
                   text="Master/PhD",
                   showarrow=False)

fig.add_annotation(x=0.075, y=0.4,
                   text=f"{res.secondary['male']:.1f}%",
                   showarrow=False)

fig.add_annotation(x=0.275, y=0.4,
                   text=f"{res['bachelor/tertiary']['male']:.1f}%",
                   showarrow=False)

fig.add_annotation(x=0.51, y=0.4,
                   text=f"{res['master/doctorate']['male']:.1f}%",
                   showarrow=False)

fig.add_annotation(x=0.075, y=0.97,
                   text=f"{res.secondary['female']:.1f}%",
                   showarrow=False)

fig.add_annotation(x=0.275, y=0.97,
                   text=f"{res['bachelor/tertiary']['female']:.1f}%",
                   showarrow=False)

fig.add_annotation(x=0.51, y=0.97,
                   text=f"{res['master/doctorate']['female']:.1f}%",
                   showarrow=False)

fig.add_annotation(x=-0.07, y=0.80,
                   text="Female",
                   showarrow=False)
fig.add_annotation(x=-0.05, y=0.20,
                   text="Male",
                   showarrow=False)
fig.show()